# Leveraging SQLAlchemy ORM to Store and Retrieve MLB Stats

## Table of Contents

[Part 1: Exploring the MLB API](#part-1)
- [1a. Install and Import](#part-1a)
- [1b. Get GamePks](#part-1b)
- [1c. The 'Game' Endpoint](#part-1c)

---

### The SQLAlchemy Object Relational Mapper automatically constructs higher-level SQL and automates persistence of python objects.
We're going to query the MLB API using a python wrapper created by Todd Roberts and store the information in a SQLite database for future analysis. 

---

<a id='part-1'></a>

## Part 1: Exploring the MLB API
Todd Roberts' python wrapper is part of the python package index. You can find more information [here](https://pypi.org/project/MLB-StatsAPI/) or on [GitHub](https://github.com/toddrob99/MLB-StatsAPI).

<a id='part-1a'></a>

First, we have to install it and import it.

In [1]:
import sys
#pip install 
#!{sys.executable} -m pip install MLB-StatsAPI

import statsapi as mlb

Todd was nice enough to give us several convenient functions for accessing the API's endpoints. The most flexible/powerful of these is the get() function that takes in an endpoint and returns the raw JSON response from the MLB Stats API. You can find a dictionary with the endpoint configuration by accessing the ENDPOINTS global variable. To get notes for a given endpoint, use the notes() method.

In [2]:
list(mlb.ENDPOINTS.keys())[:10]

['attendance',
 'awards',
 'conferences',
 'divisions',
 'draft',
 'game',
 'game_diff',
 'game_timestamps',
 'game_changes',
 'game_contextMetrics']

In [3]:
print(mlb.notes('game'))

Endpoint: game 
All path parameters: ['ver', 'gamePk']. 
Required path parameters (note: ver will be included by default): ['ver', 'gamePk']. 
All query parameters: ['timecode', 'hydrate', 'fields']. 
Required query parameters: None. 
The hydrate function is supported by this endpoint. Call the endpoint with {'hydrate':'hydrations'} in the parameters to return a list of available hydrations. For example, statsapi.get('schedule',{'sportId':1,'hydrate':'hydrations','fields':'hydrations'})



<a id='part-1b'></a>

#### Get GamePks

In [4]:
from datetime import datetime as dt
import os,re,csv
from os import walk

#dates from the 'season' endpoint are returned in a different format than what we need to query the API
#we'll use this function to take care of that in a moment
def convert_date(date):
    date = dt.strptime(date,"%Y-%m-%d")
    convertedDate = dt.strftime(date,"%m/%d/%Y")
    return convertedDate

def get_gamePks(seasons,target_directory=None):
    """
    Takes in a list of seasons as strings representing their year e.g. ['2018','2019']
    Queries the MLB API to find gamePks for each season and writes them to CSV files
    if a target directory for the gamePks is not specified, a directory called 'gamePks'
    will be added to the current directory. 
    """
    if target_directory:
        gamePks_path = target_directory
    else:
        #create a directory to store CSVs
        try:
            os.mkdir(os.getcwd()+'/gamePks')
        except FileExistsError:
            pass
        gamePks_path=os.getcwd()+'/gamePks'
    
    #walk the gamePks directory to see if we've already added any seasons
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    years = [re.findall('[^.csv]+',x) for x in f]
    already_added = [item for sublist in years for item in sublist if item[0] in ['1','2']]
    seasons = list(set(seasons)-set(already_added))
    
    #query the API to get start dates and end dates for all seasons
    all_seasons = mlb.get('seasons',{'sportId':1,'all':True})['seasons']
    
    #filter out the ones we don't care about right now
    seasons = list(filter(lambda x: x['seasonId'] in seasons,all_seasons))
    
    gamePks = {}
    for season in seasons:  
        year = season['seasonId']
        startDate = convert_date(season['seasonStartDate'])
        endDate = convert_date(season['seasonEndDate'])
        
        #returns a list of dicts for each date in the range
        #each dict has a 'games' key with a list of dicts for each game in that day as values
        dates = mlb.get('schedule',{'sportId':1,'startDate':startDate,'endDate':endDate})['dates']
        
        #for each date, and for each game in that date, get the gamePk 
        gamePks[year]= [ game['gamePk'] 
                                          for date in dates 
                                          for game in date['games'] ]
        #store the gamePks as CSVs
        with open(gamePks_path + f"/{year}.csv", 'w',newline='') as myfile:
            wr = csv.writer(myfile,quoting=csv.QUOTE_ALL)
            wr.writerow(gamePks[year])

def read_gamePks():
    gamePks_path = os.curdir+'/gamePks'
    f = []
    for (dirpath, dirnames, filenames) in walk(gamePks_path):
        f.extend(filenames)
        break
    pk_paths = [gamePks_path + '/' + x for x in f if x[0]!= '.']
    
    gamePks = {}
    for path in pk_paths:
        season = re.findall('/gamePks/([^.csv]+)',path)
        with open(path, 'r') as f:
            reader = csv.reader(f)
            seasonPks = list(reader)
        gamePks[season[0]] = [item for sublist in seasonPks for item in sublist]
    return gamePks

In [5]:
get_gamePks([str(x) for x in range(2008,2020)])   

gamePks=read_gamePks()

<a id='part-1c'></a>

#### Explore the 'Game' Endpoint

Let's pick a gamePk at random to see what's inside the 'game' endpoint. There is a TON of information stored in nested dictionaries returned from the API query. Since our goal is to store this information in a SQL database, our aim is to organize the information into [first normal form](https://www.essentialsql.com/get-ready-to-learn-sql-8-database-first-normal-form-explained-in-simple-english/).

From this single result, we'll be able to start building [normalized SQL tables](https://www.essentialsql.com/get-ready-to-learn-sql-database-normalization-explained-in-simple-english/) for games, teams, venues, players, plays, and pitches. Let's start with the games table. 

In [6]:
temp_pk=gamePks['2019'][501]
print(temp_pk) 
game_result = mlb.get('game',{'gamePk':temp_pk})

565812


In [7]:
game_result.keys()

dict_keys(['copyright', 'gamePk', 'link', 'metaData', 'gameData', 'liveData'])

In [8]:
gameData = game_result['gameData']
gameData.keys()

dict_keys(['game', 'datetime', 'status', 'teams', 'players', 'venue', 'weather', 'review', 'flags', 'alerts', 'probablePitchers', 'officialScorer', 'primaryDatacaster'])

In [9]:
game = gameData['game']
game

{'pk': 565812,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/04/26/pitmlb-lanmlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-565812-2019-04-26',
 'season': '2019',
 'seasonDisplay': '2019'}

The dictionary above, nested 2 layers deep into the original API result, provides us with a good starting point. But we're goint to add some additional information to make our games table more informative

In [10]:
gameData['datetime']

{'dateTime': '2019-04-27T02:10:00Z',
 'originalDate': '2019-04-26',
 'dayNight': 'night',
 'time': '7:10',
 'ampm': 'PM'}

In [11]:
gameData['weather']

{'condition': 'Clear', 'temp': '61', 'wind': '2 mph, Varies'}

In [12]:
gameData['venue']['timeZone']

{'id': 'America/Los_Angeles', 'offset': -8, 'tz': 'PST'}

In [13]:
gameData['status']

{'abstractGameState': 'Final',
 'codedGameState': 'F',
 'detailedState': 'Final',
 'statusCode': 'F',
 'abstractGameCode': 'F'}

In [14]:
#keys to add
keys_to_add = ['dateTime',
          'originalDate',
          'condition',
          'temp','wind','tz']
#dictionaries from which to add them
dicts = [gameData['datetime'],
         gameData['weather'],
         gameData['venue']['timeZone']
        ]

for k in keys_to_add:
    for d in dicts:
        try:
            game[k]=d[k]
        except KeyError:
            continue
#'seasonDisplay' key:value seems to be redundant
del game['seasonDisplay']
game

{'pk': 565812,
 'type': 'R',
 'doubleHeader': 'N',
 'id': '2019/04/26/pitmlb-lanmlb-1',
 'gamedayType': 'P',
 'tiebreaker': 'N',
 'gameNumber': 1,
 'calendarEventID': '14-565812-2019-04-26',
 'season': '2019',
 'dateTime': '2019-04-27T02:10:00Z',
 'originalDate': '2019-04-26',
 'condition': 'Clear',
 'temp': '61',
 'wind': '2 mph, Varies',
 'tz': 'PST'}

In [15]:
gameData['probablePitchers']

{'away': {'id': 502042,
  'fullName': 'Archer, Chris',
  'link': '/api/v1/people/502042'},
 'home': {'id': 547943,
  'fullName': 'Ryu, Hyun-Jin',
  'link': '/api/v1/people/547943'}}

In [16]:
def get_game(api_call):
    gameData = api_call['gameData']
    dateTime = gameData['datetime']
    game = gameData['game']
    weather = gameData['weather']
    timeZone = gameData['venue']['timeZone']
    status = gameData['status']
    probablePitchers = gameData['probablePitchers']
    
    keys_to_add = ['dateTime','originalDate',
                   'condition','temp','wind',
                   'tz','detailedState'
                  ]
    dicts = [weather,dateTime,timeZone,status]
    for k in keys_to_add:
        for d in dicts:
            try:
                game[k]=d[k]
            except KeyError:
                continue
    #'seasonDisplay' key:value seems to be redundant
    del game['seasonDisplay']
    
    home_team = gameData['teams']['home']
    away_team = gameData['teams']['away']
    
    game['homeTeam_id'] = home_team['id']
    game['awayTeam_id'] = away_team['id']
    
    game['venue_id'] = gameData['venue']['id']
    
    for team in ['home','away']:
        try:
            game[f"{team}_probablePitcher"]=probablePitchers[team]['id']
        except KeyError:
            pass
    
    #format the dateTime and originalDate
    fmt = "%Y-%m-%dT%H:%M:%SZ" 
    game['dateTime'] = dt.strptime(game['dateTime'],fmt)
    fmt = "%Y-%m-%d"
    game['originalDate'] = dt.strptime(game['originalDate'],fmt).date()
    
    return game
api_call = mlb.get('game',{'gamePk':temp_pk})
game = get_game(api_call)

## API calls
What's the best way to automate API calls when needed?

In [17]:
def api_calls(gamePks):
    return ( mlb.get('game',{'gamePk':gamePk}) for gamePk in gamePks )

In [18]:
calls = api_calls(gamePks['2019'][500:600])

In [19]:
#test = [get_game(x) for x in calls]

<a id='part-2'></a>

## Introducing SQL Alchemy

In [20]:
import sqlalchemy
from sqlalchemy import create_engine,PrimaryKeyConstraint,UniqueConstraint

In [21]:
class MyDatabase:
    # http://docs.sqlalchemy.org/en/latest/core/engines.html
    """
    Custom class for instantiating a SQL Alchemy connection. Configured here for SQLite, but intended to be flexible.
    Credit to Medium user Mahmud Ahsan:
    https://medium.com/@mahmudahsan/how-to-use-python-sqlite3-using-sqlalchemy-158f9c54eb32
    """
    DB_ENGINE = {
       'sqlite': 'sqlite:////{DB}'
    }

    # Main DB Connection Ref Obj
    db_engine = None
    def __init__(self, dbtype, username='', password='', dbname='',path=os.getcwd()+'/'):
        dbtype = dbtype.lower()
        if dbtype in self.DB_ENGINE.keys():
            engine_url = self.DB_ENGINE[dbtype].format(DB=path+dbname)
            self.db_engine = create_engine(engine_url)
            print(self.db_engine)
        else:
            print("DBType is not found in DB_ENGINE")
db=MyDatabase('sqlite',dbname='mlb.db')

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb_sqlite/blog_posts/mlb.db)


In [22]:
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

from sqlalchemy import Table,Column,Integer,String,DateTime,Date,Boolean

class Game(Base):
    __tablename__ = 'games'
    __table_args__ = (PrimaryKeyConstraint('id','detailedState',sqlite_on_conflict='IGNORE'),
                      {'extend_existing': True})
    
    pk = Column(Integer)
    type = Column(String(1))
    doubleHeader = Column(String(1))
    id = Column(String(150))
    gamedayType = Column(String(1))
    tiebreaker = Column(String(1))
    gameNumber = Column(Integer)
    calenderEventId = Column(String(50))
    season = Column(Integer)
    
    dateTime = Column(DateTime)
    originalDate = Column(Date)
    
    detailedState = Column(String(12))
    
    homeTeam_id = Column(Integer)
    awayTeam_id = Column(Integer)
    
    condition = Column(String(25))
    temp = Column(Integer)
    wind = Column(String(50))
    
    venue_id = Column(Integer)
    
    home_probablePitcher = Column(Integer)
    away_probablePitcher = Column(Integer)
    
    def __repr__(self): 
        return "<Game(pk='%s',id='%s')>" % (
                        self.pk, self.id)
    
    def __init__(self,dictionary):
        for k,v in dictionary.items():
            setattr(self,k,v)

In [23]:
Base.metadata.create_all(db.db_engine)

In [24]:
game_record = Game(game)

In [25]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

In [26]:
game_record

<Game(pk='565812',id='2019/04/26/pitmlb-lanmlb-1')>

In [27]:
session.add(game_record)

session.commit()

session.query(Game).all()

[<Game(pk='565812',id='2019/04/26/pitmlb-lanmlb-1')>]

Now we're going to move on to plays. We're going to set up a one-to-many relationship between games and plays, so we'll need to be able to link the two when we get to the SQLAlchemy stage. With that in mind, we'll start exploring the information embedded in the 'liveData' key of the original API result

'allPlays' gives us a list of dictionaries with information about every play. Remember that we're aiming for first normal form, so we don't want any information about pitches, players, or runners at this time. Those will be contained in their own respective tables

In [28]:
def string_to_dateTime(string):
    fmt="%Y-%m-%dT%H:%M:%S.%fZ"
    return dt.strptime(string,fmt)
string_to_dateTime('2019-04-26T20:59:22.000Z')

datetime.datetime(2019, 4, 26, 20, 59, 22)

In [29]:
# def parse_play(play):
#     play_dict = play['result']
#     play_dict.update(play['about'])
#     play_dict.update(play['count'])
    
#     for t in ['startTime','endTime']:
#         play_dict[t]=string_to_dateTime(play_dict[t])
        
#     for player in ['batter','pitcher']:
#         try:
#             play_dict[f"{player}_id"] = play['matchup'][player]['id']
#         except:
#             pass
#     return play_dict

In [30]:
# def get_plays(API_result):
#     #foreign key references games table
#     gamePk={'gamePk':API_result['gamePk']}
    
#     allPlays = API_result['liveData']['plays']['allPlays']
#     plays = [parse_play(play) for play in allPlays] 
#     [play.update(gamePk) for play in plays]
#     return plays

In [31]:
def flatten_dicts(dictionary):
    """
    recursively flatten a dictionary of dictionaries
    """
    #base case 
    if dict not in [type(x) for x in dictionary.values()]:
        return dictionary
    else:
        for key, value in dictionary.items():
            if type(value)==dict:
                temp_dict = dictionary.pop(key)
                for k,v in temp_dict.items():
                    dictionary[f"{key}_{k}"]=v
                return flatten_dicts(dictionary)



In [32]:
def get_plays(API_result):
    #foreign key references games table
    gamePk={'gamePk':API_result['gamePk']}
    
    allPlays = API_result['liveData']['plays']['allPlays']
    keys = ['result','about','count']
    plays = []
    matchups = []
    hotColdZones = []
    for play in allPlays:
        play_dict = {k:v for k,v in zip(keys,[play[key] for key in keys])}.copy()
        plays.append(flatten_dicts(play_dict))
        matchup = play.pop('matchup')
        #foreign keys to play 'atBatIndex', 'playEndTime'
        fks=['atBatIndex', 'playEndTime']
            
        matchup.update(
            {k:v for k,v in zip(fks,[play[fk] for fk in fks])}
        )
        for x in ['pitcher','batter']:
            hotColdZone =  matchup.pop(f"{x}HotColdZones")
            for zone in hotColdZone:
                try:
                    zone.update(
                        matchup.pop(f'{x}HotColdZoneStats')
                    )
                except KeyError:
                    pass
                zone.update(
                    {k:v for k,v in zip(fks,[play[fk] for fk in fks])}
                )
                zone.update({'type':x})
                hotColdZones.append(flatten_dicts(zone))
        
        matchups.append(flatten_dicts(matchup))
    
    [hc.update(gamePk) for hc in hotColdZones]        
    [m.update(gamePk) for m in matchups]
    [p.update(gamePk) for p in plays]
    
    hotColdStats = []
    for hc in hotColdZones:
        try:
            stats = hc.pop('stats')
        except KeyError:
            continue
        for stat in stats:
            stat.update(gamePk)
            stat.update(
                {k:v for k,v in zip(fks,[play[fk] for fk in fks])}
            )
            hotColdStats.append(flatten_dicts(stat))
        
    return plays,matchups,hotColdZones,hotColdStats
        
        

In [33]:
def get_pitches(API_result):
    #foreign key references games table
    gamePk={'gamePk':API_result['gamePk']}
    
    allPlays = API_result['liveData']['plays']['allPlays']
    
    pitches = []
    for play in allPlays:
        for i in play['pitchIndex']:
            pitch = play['playEvents'][i]
            #foreign keys to play 'atBatIndex', 'playEndTime'
            fks=['atBatIndex', 'playEndTime']
            
            pitch.update(
                {k:v for k,v in zip(fks,[play[fk] for fk in fks])}
            )
            
            pitches.append(flatten_dicts(pitch))
    
    [p.update(gamePk) for p in pitches]
    return pitches

In [34]:
def get_runners(API_result):
    #foreign key references games table
    gamePk={'gamePk':API_result['gamePk']}
    
    allPlays = API_result['liveData']['plays']['allPlays']
    
    runners = []
    credits = []
    for play in allPlays:
        for i in play['runnerIndex']:
            runner = play['runners'][i]
            
            fks=['atBatIndex', 'playEndTime']
            
            runner.update(
                {k:v for k,v in zip(fks,[play[fk] for fk in fks])}
            )
            try:
                temp_credits = runner.pop('credits')
                
                for credit in temp_credits:
                    credit.update({k:v for k,v in zip(fks,[play[fk] for fk in fks])})
                
                    credits.append(flatten_dicts(credit))
            except KeyError:
                pass 
            runners.append(flatten_dicts(runner))
    
    [r.update(gamePk) for r in runners]
    return runners,credits

In [35]:
def get_actions(API_result):
    #foreign key references games table
    gamePk={'gamePk':API_result['gamePk']}
    
    allPlays = API_result['liveData']['plays']['allPlays']
    
    actions = []
    for play in allPlays:
        for i in play['actionIndex']:
            action = play['playEvents'][i]
            #foreign keys to play 'atBatIndex', 'playEndTime'
            fks=['atBatIndex', 'playEndTime']
            
            action.update(
                {k:v for k,v in zip(fks,[play[fk] for fk in fks])}
            )
            
            actions.append(flatten_dicts(action))
    
    [a.update(gamePk) for a in actions]
    return actions

In [36]:
def get_players(API_result):
    #fk for game_player_link
    gamePk=API_result['gamePk']
    
    players = API_result['gameData']['players']
    players = [flatten_dicts(players[player_id]) for player_id in players.keys()]
    
    game_player_links = []
    for player in players:
        link = {'player':player['id'],'gamePk':gamePk}
        game_player_links.append(link)
    
    return players,game_player_links

In [37]:
players,game_player_links = get_players(api_call)

In [38]:
def get_teams(API_result):
    #fk for game_team_link
    gamePk=API_result['gamePk']
    teams_dict = API_result['gameData']['teams']
    
    teams = []
    links = []
    team_records = []
    for key in ['home','away']:
        team = teams_dict[key]
        
        team_record = team.pop('record')
        team_record.update({'gamePk':gamePk})
        team_records.append(team_record)
        
        teams.append(flatten_dicts(team))
        
        link = {'gamePk':gamePk,
                'team_id':team['id'],
                'home_away':key}
        links.append(link)
        
    return teams, links, team_records

In [39]:
def get_venue(API_result):
    venue = API_result['gameData']['venue']
    return flatten_dicts(venue)

In [40]:
import pandas as pd

test_gamePks = gamePks['2019'][::300]

In [41]:
def parse_games(gamePks):
    api_results=api_calls(gamePks)
    tables = ['games','plays','pitches',
              'runners','credits','actions',
              'teams','venues','game_team_links',
              'game_player_links','team_records',
              'players','matchups','hotColdZones','hotColdStats']
    dfs = {}
    for table in tables:
        dfs[table]=[]
    for result in api_results:
        dfs['games'].append(get_game(result))
        dfs['venues'].append(get_venue(result))
        dfs['pitches'].extend(get_pitches(result))
        dfs['actions'].extend(get_actions(result))
        
        plays,matchups,hotColdZones,hotColdStats = get_plays(result)
        dfs['plays'].extend(plays)
        dfs['matchups'].extend(matchups)
        dfs['hotColdZones'].extend(hotColdZones)
        dfs['hotColdStats'].extend(hotColdStats)
        
        runners,credits = get_runners(result)
        dfs['runners'].extend(runners)
        dfs['credits'].extend(credits)
        
        teams, game_team_links, team_records = get_teams(result)
        dfs['teams'].extend(teams)
        dfs['game_team_links'].extend(game_team_links)
        dfs['team_records'].extend(team_records)
        
        players,game_player_links = get_players(result)
        dfs['players'].extend(players)
        dfs['game_player_links'].extend(game_player_links)
    for key in dfs.keys():
        dfs[key]=pd.DataFrame.from_records(dfs[key])
    return dfs
        
    

In [42]:
import joblib

In [43]:
joblib.dump(api_call,'pickle.pkl')

['pickle.pkl']

In [44]:
dfs = parse_games(test_gamePks)

In [45]:
len(gamePks['2019'])*500

1308000

In [46]:
dfs['matchups']

,atBatIndex,playEndTime,batter_id,batter_fullName,batter_link,batSide_code,batSide_description,pitcher_id,pitcher_fullName,pitcher_link,...,gamePk,postOnFirst_id,postOnFirst_fullName,postOnFirst_link,postOnSecond_id,postOnSecond_fullName,postOnSecond_link,postOnThird_id,postOnThird_fullName,postOnThird_link
0,0,2019-03-20T09:36:53.000Z,543829,Dee Gordon,/api/v1/people/543829,L,Left,571666,Mike Fiers,/api/v1/people/571666,...,566083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2019-03-20T09:38:55.000Z,571745,Mitch Haniger,/api/v1/people/571745,R,Right,571666,Mike Fiers,/api/v1/people/571666,...,566083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2019-03-20T09:39:51.000Z,457803,Jay Bruce,/api/v1/people/457803,L,Left,571666,Mike Fiers,/api/v1/people/571666,...,566083,457803.0,Jay Bruce,/api/v1/people/457803,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2019-03-20T09:42:20.000Z,429665,Edwin Encarnacion,/api/v1/people/429665,R,Right,571666,Mike Fiers,/api/v1/people/571666,...,566083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2019-03-20T09:45:38.000Z,657656,Ramon Laureano,/api/v1/people/657656,R,Right,594835,Marco Gonzales,/api/v1/people/594835,...,566083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
683,70,2019-09-18T01:31:32.000Z,592743,Andrelton Simmons,/api/v1/people/592743,R,Right,664856,Chance Adams,/api/v1/people/664856,...,567531,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
684,71,2019-09-18T01:34:10.000Z,665120,Jared Walsh,/api/v1/people/665120,L,Left,664856,Chance Adams,/api/v1/people/664856,...,567531,665120.0,Jared Walsh,/api/v1/people/665120,NaN,NaN,NaN,NaN,NaN,NaN
685,72,2019-09-18T01:37:33.000Z,642136,Matt Thaiss,/api/v1/people/642136,L,Left,664856,Chance Adams,/api/v1/people/664856,...,567531,642136.0,Matt Thaiss,/api/v1/people/642136,665120.0,Jared Walsh,/api/v1/people/665120,NaN,NaN,NaN
686,73,2019-09-18T01:40:20.000Z,641684,Michael Hermosillo,/api/v1/people/641684,R,Right,664856,Chance Adams,/api/v1/people/664856,...,567531,642136.0,Matt Thaiss,/api/v1/people/642136,665120.0,Jared Walsh,/api/v1/people/665120,NaN,NaN,NaN


In [47]:
dfs['hotColdStats']['splits'][0]

[{'season': '2019',
  'stat': {'name': 'battingAverage',
   'zones': [{'zone': '01',
     'color': 'rgba(6, 90, 238, .55)',
     'temp': 'cold',
     'value': '.000'},
    {'zone': '02',
     'color': 'rgba(214, 41, 52, .55)',
     'temp': 'hot',
     'value': '.400'},
    {'zone': '03',
     'color': 'rgba(214, 41, 52, .55)',
     'temp': 'hot',
     'value': '.600'},
    {'zone': '04',
     'color': 'rgba(214, 41, 52, .55)',
     'temp': 'hot',
     'value': '.600'},
    {'zone': '05',
     'color': 'rgba(150, 188, 255, .55)',
     'temp': 'cool',
     'value': '.250'},
    {'zone': '06',
     'color': 'rgba(150, 188, 255, .55)',
     'temp': 'cool',
     'value': '.222'},
    {'zone': '07',
     'color': 'rgba(214, 41, 52, .55)',
     'temp': 'hot',
     'value': '.600'},
    {'zone': '08',
     'color': 'rgba(234, 147, 153, .55)',
     'temp': 'warm',
     'value': '.333'},
    {'zone': '09',
     'color': 'rgba(6, 90, 238, .55)',
     'temp': 'cold',
     'value': '.154'},
    {'z

In [48]:
hotCold=dfs['hotColdZones']

In [49]:
hotCold[hotCold['stats'].isna()==False][['playEndTime','atBatIndex','stats']].to_records()[0]

KeyError: 'stats'

In [ ]:
dfs['hotColdZones']

In [ ]:
matchups = [get_plays(x)[1] for x in api_calls(test_gamePks)]

In [ ]:
matchups = [item for sublist in matchups for item in sublist]

In [ ]:
dfs.keys()

In [ ]:
dfs['pitches'].columns

In [ ]:
dfs['pitches'].info()

In [ ]:
pitches = dfs['pitches']

In [ ]:
pitches['details_description'].value_counts()

In [ ]:
pitches['details_isInPlay'].sum()

In [ ]:
pitches[pitches['playId'].isna()]

In [ ]:
def unique_columns(df):
    """
    returns names for columns that have all unique values
    """
    return [x for x in df.columns if len(df[x].unique())==df.shape[0]]

In [ ]:
pitches_df = pd.DataFrame.from_records(pitches)

In [ ]:
pitches_df.columns

In [ ]:
pitches_df['details_description'].unique()

In [ ]:
pitches_df[pitches_df['details_description']=='In play, out(s)']['hitData_totalDistance']

In [ ]:
unique_columns(pitches_df)

In [ ]:
plays_df.shape==len(pitches_df['atBatIndex'].unique())

In [ ]:
pitches_df.info()

In [ ]:
pitches_df['hitData_hardness'].unique()